In [1]:
import pandas as pd
import py4vasp as pv
import numpy as np
import re 

In [2]:
calc = pv.Calculation.from_file('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mlff_otf_300k.h5')

In [3]:
calc.structure[4000].read()

{'lattice_vectors': array([[ 9.73349181,  0.        ,  0.        ],
        [ 0.        ,  9.73349181, -0.        ],
        [-0.        , -0.        ,  9.73349181]]),
 'positions': array([[ 6.36893599e-02,  1.27089341e-01,  2.49564921e-01],
        [ 9.50137721e-01,  8.17935790e-01,  1.82230971e-01],
        [ 9.67113387e-01,  3.28792599e-01,  6.67225492e-01],
        [ 2.33733357e-02,  6.53958203e-01,  7.30215871e-01],
        [ 4.09300839e-01,  5.28419666e-02,  2.59731391e-01],
        [ 1.42481045e-01,  9.54852050e-01,  8.29964802e-01],
        [ 7.23886038e-01,  9.47992628e-01,  3.71908459e-01],
        [ 8.27215870e-01,  4.00210327e-02,  7.80453875e-01],
        [ 2.07774064e-01,  2.29582284e-01,  1.73790287e-02],
        [ 8.07523891e-01,  1.84669752e-01,  9.94327331e-01],
        [ 2.75411965e-01,  6.96946721e-01,  9.42889383e-01],
        [ 6.28801809e-01,  7.21843019e-01,  3.29769793e-02],
        [ 3.30380689e-01,  5.33182480e-01,  7.56350142e-01],
        [ 1.19731060e-01, 

In [4]:
energies = [calc.energy[0].read()['total energy   ETOTAL']]
energies_square = []
energy_square = []

for energy in calc.energy[1:].read()['total energy   ETOTAL']:
    energies.append(energy)
    energy_square.append(np.mean(energies)**2)
    energies_square.append(np.mean(np.square(energies)))

In [5]:
t, k = 300, 8.617333262E-5

df = pd.DataFrame({
    '<E^2>': energies_square,
    '<E>^2>': energy_square,
    'C_v [eV/K]': 1/(k*t**2)*(np.subtract(energies_square,energy_square))
})

In [6]:
import plotly.express as px

px.scatter(df)

In [7]:
df['C_v [1/K]'].tail(1)

KeyError: 'C_v [1/K]'

In [ ]:
def get_cv(file_path:str, t):
    k=8.617333262E-5 # Boltzman in eV/K
    calc = pv.Calculation.from_file(file_path)
    energies = calc.energy[:].read()['total energy   ETOTAL']
    energies_square = np.mean(np.square(energies))
    energy_square = np.mean(energies)**2

    c_v = 1/(k*t**2)*(energies_square-energy_square)
    print(f'Calculated C_v value for {file_path}: {c_v}')
    return c_v

In [ ]:
folder = '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv'

data_mlff = pd.DataFrame({
    'Temperature [K]': [300,800,300,800,300,800,300,800],
    'Name': ['aimd_300k','aimd_800k', 'mlff_fake_300k', 'mlff_fake_800k', 'mlff_otf_300k','mlff_otf_800k', 'mlff_otf_npt_300k', 'mlff_otf_npt_800k'],
    'Type': ['AIMD','AIMD','MLFF Fake', 'MLFF Fake', 'MLFF OTF', 'MLFF OTF', 'MLFF NPT', 'MLFF NPT'],
    'C_v [eV/K]': 0
})

In [ ]:
data_mlff['C_v [eV/K]'][1]

0

In [ ]:
data_mlff.loc[1,'C_v [eV/K]']

-9.352098950958394

In [ ]:
for i, name in enumerate(data_mlff['Name']):
    data_mlff.loc[i,'C_v [eV/K]'] = get_cv(f'{folder}/{name}.h5',data_mlff['Temperature [K]'][i])

Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/aimd_300k.h5: 0.0010088016649774897
Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/aimd_800k.h5: 0.002395570541560055
Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mlff_fake_300k.h5: 2.8375446879712154e-08
Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mlff_fake_800k.h5: 8.935489999951512e-09
Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mlff_otf_300k.h5: 3.477085193753791e-08
Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mlff_otf_800k.h5: 1.2032634001706446e-08
Calculated C_v value for /Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mlff_otf_npt_300k.h5: 3.158769

In [ ]:
# mtp
data_mtp = np.loadtxt('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Compare_ML/MD_Cv/mtp_300k')

In [ ]:
df_mtp = pd.DataFrame(data_mtp, columns=['Step','Temperature', 'Volume', 'Pressure', 'Kinetic energy', 'Potential energy', 'Total energy', 'Fmax'])

In [ ]:
energies = [df_mtp['Total energy'][0]]
energies_square = []
energy_square = []

for energy in df_mtp['Total energy']:
    energies.append(energy)
    energy_square.append(np.mean(energies)**2)
    energies_square.append(np.mean(np.square(energies)))

In [ ]:
t, k = 300, 8.617333262E-5

df = pd.DataFrame({
    '<E^2>': energies_square,
    '<E>^2>': energy_square,
    'C_v [eV/K]': 1/(k*t**2)*(np.subtract(energies_square,energy_square))
})

In [ ]:
import plotly.express as px

px.scatter(df)

In [ ]:
t, k = 300, 8.617333262E-5
#c_v = 1/(k*t**2)*(energies_square-energy_square)
c_v = 1/(k*t**2)*(np.mean(np.square(df_mtp['Total energy']))- np.mean(df_mtp['Total energy'])**2)